In [1]:
%matplotlib notebook
import numpy as np
import math
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display
import scipy.stats

### One-sample t-test

In this example we show an interactive demo of how to perform a hypothesis test of the population mean based on the t-distribution. We use the t-distribution to test the population mean when the sampling distribution of the point estimate of interest (in this case the sample mean) is nearly normally distributed and the sample is small.

The thicker tails of the t-distribution, compared to those of the normal distribution, is the correction factor we need to compensate for the small size of the sample. 

We follow the same process followed for the case of hypothesis testing based on the normal model, but using the t-distribution instead. The T-score is the equivalent of the Z-score in this context, and it is calculated exactly in the same way. 

The one-sample t-test is commonly applied to large samples as well because the results are then very similar to those of hypothesis testing based on the normal model. This is because the t-distribution becomes practically indistinguishible from the normal distribution when the value of the degrees of freedom is higher than 30. 

### One-sample t-test (paired data)

We say that we have paired data when we work with two datasets in which one observation from one dataset corresponds to exactly another single observation from the second dataset. To test whether there is a difference between the mean of the two populations we create a derived dataset by substracting each observation from the first sample to the corresponding observation from the second sample, and we perform a one-sample t-test in which the null hypothesis is that the mean of the derived dataset is equal to zero.

### Two-sample t-test

The two-sample t-test is aimed at determining whether there is a difference between the mean of two populations when data is not paired. The point estimate is calculated from a sample from each population by substracting their sample means. It is aimed at extracting conclusions from experiments: "is there a difference between the treatment and the control groups?". In this setting, the null hypothesis considers that the value of the difference of means is zero. 

In this example we will assume that we have some evidence that the standard deviation of the two populations is approximately the same. That's the reason why the demo only lets you set the value of the standard deviation of one of the samples and then creates a random very similar standard deviation value for the second sample. 

Given this assumption we can use the pooled variance method to improve the estimation of the standard deviation of the sampling distributions. The pooled variance method also lets us assign a higher value to the degrees of freedom parameter of the t-distribution, therefore letting us apply the test on a t-distribution that looks closer to a normal distribution. 